# Collaborative filtering practice

In this homework you will test different collaborative filtering (CF) approaches on famous Movielens dataset.

In class we implemented item2item CF, so this time let's use **user2user** approach.

## Task 0: Dataset (5 points)

Load [movielens](https://grouplens.org/datasets/movielens/) dataset using [scikit surprise](https://surprise.readthedocs.io/en/stable/dataset.html)

Split dataset to train and validation parts.

Don't forget to encode users and items from 0 to maximum!

In [ ]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from surprise import Dataset, Reader
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import os

from sklearn.preprocessing import LabelEncoder
from scipy.stats import pearsonr

from functools import lru_cache
from typing import Tuple, Callable, List

In [3]:
def ids_encoder(ratings):
    users = sorted(ratings['userid'].unique())
    items = sorted(ratings['itemid'].unique())

    uencoder = LabelEncoder()
    iencoder = LabelEncoder()

    uencoder.fit(users)
    iencoder.fit(items)

    ratings['userid'] = uencoder.transform(ratings['userid'])
    ratings['itemid'] = iencoder.transform(ratings['itemid'])

    return ratings, uencoder, iencoder

def normalize_ratings(ratings: pd.DataFrame) -> pd.DataFrame:
    mean = ratings.groupby(by='userid', as_index = False)['rating'].mean()
    norm_ratings = pd.merge(ratings, mean, suffixes = ('','_mean'), on='userid')
    norm_ratings['norm_rating'] = norm_ratings['rating'] - norm_ratings['rating_mean']
    return norm_ratings[ratings.columns.tolist() + ['norm_rating']]

def prepare_data(df):
    df, uencoder, iencoder = ids_encoder(df)
    df = normalize_ratings(df)
    return df, uencoder, iencoder

In [9]:
if not (os.path.exists("recsys.zip") or os.path.exists("recsys")):
    os.system("wget https://github.com/nzhinusoftcm/review-on-collaborative-filtering/raw/master/recsys.zip")
    os.system("unzip recsys.zip")
from recsys.datasets import ml100k

ratings, movies = ml100k.load()

ratings_train, ratings_test = train_test_split(ratings, test_size = 0.25, random_state = 42)
data_train, uencoder_train, iencoder_train = prepare_data(ratings_train)
data_test, uencoder_test, iencoder_test = prepare_data(ratings_test)

## Task 1: Similarities (5 points each)

You need to implement 3 similarity functions:
1. Dot product (intersection)
1. Jaccard index (intersection over union)
1. Pearson correlation
1. Pearson correlation with decreasing coefficient

In [5]:
class SimilarityFunctions():
    '''Class of functions to calculate similarity between items'''
    def __init__(self):
        pass
    def sim_dot(self, x: np.array, y: np.array) -> float:
        common_items = np.intersect1d(x[:, 1], y[:, 1])
        common_I_i = x[np.isin(x[:, 1], common_items)]
        common_I_j = y[np.isin(y[:, 1], common_items)]
        if len(common_items) > 0:
            assert sorted(common_I_i[:, 1]) == sorted(common_I_j[:, 1])
            x = common_I_i[:, 3]
            y = common_I_j[:, 3]
            return np.dot(x, y)
        return -1.0


    def sim_jacc(self, x: np.array, y: np.array) -> float:
        union_items = np.union1d(x[:, 1], y[:, 1])
        common_items = np.intersect1d(x[:, 1], y[:, 1])
        return len(common_items)/len(union_items)

    def sim_pearson(self, x: np.array, y: np.array) -> float:
        common_items = np.intersect1d(x[:, 1], y[:, 1])
        common_I_i = x[np.isin(x[:, 1], common_items)]
        common_I_j = y[np.isin(y[:, 1], common_items)]
        if len(common_items) > 0:
            assert sorted(common_I_i[:, 1]) == sorted(common_I_j[:, 1])
            x = common_I_i[:, 3]
            y = common_I_j[:, 3]
            if np.linalg.norm(x) == 0 or np.linalg.norm(y) == 0:
                return 0
            if x.shape[0] == 1 or y.shape[0] == 1:
                return 1
            return pearsonr(x, y)[0]
        return -1.0

    def sim_pearson_decreasing(self, x: np.array, y: np.array) -> float:
        common_items = np.intersect1d(x[:, 1], y[:, 1])
        sim = min(1, len(common_items)/50)* SimilarityFunctions.sim_pearson(self,x, y)
        return sim

## Task 2: Collaborative filtering algorithm (5 points each)

Now you have several options to use similarities for ratings prediction:
1. Simple averaging
1. Mean corrected averaging

In [10]:
class User2UserRecomendations:
    def __init__(self, sim_fn, movies_names, mean_correct: bool = False):
        self.sim_fn = sim_fn
        self.mean_correct = mean_correct
        self.movies = movies_names

    def items_for_user(self, i: int) -> np.array:
        return self.feedbacks[self.feedbacks[:, 0] == i]

    def calculate_sim_between_two(self, i: int, j: int) -> float:
        if i == j:
            return 1.0
        I_i, I_j = self.items_for_user(i), self.items_for_user(j)
        return self.sim_fn(self, I_i, I_j)

    def calculate_pairwise_sim(self) -> Tuple[np.array, np.array]:
        nb_users = np.unique(self.feedbacks[:, 0]).size
        similarities = np.full((nb_users, nb_users), -1.0)
        np.fill_diagonal(similarities, 1)
        users = sorted(set(map(int, self.feedbacks[:, 0])))
        __import__('random').shuffle(users)
        with tqdm(total=len(users) * (len(users) - 1) // 2) as pbar:
            for i in range(len(users)):
                for j in range(i + 1, len(users)):
                    sim = self.calculate_sim_between_two(users[i], users[j])
                    similarities[users[i], users[j]] = sim
                    similarities[users[j], users[i]] = sim
                    pbar.update()
        neighbors = np.flip(np.argsort(similarities), axis=1)
        similarities = np.flip(np.sort(similarities), axis=1)
        return similarities, neighbors

    def candidate_items(self, userId: int, k: int = -1) -> Tuple[np.array, np.array]:
        I_u = self.feedbacks[self.feedbacks[:, 0] == userId]
        I_u = I_u[:, 1].astype('int')
        U_u = self.neighbors[userId, :][1:k+1]
        c = set()
        for similar_user in U_u:
            items_rated_by_similar_user = self.feedbacks[self.feedbacks[:, 0] == similar_user][:, 1].astype('int')
            c.update(items_rated_by_similar_user)
        c = list(c)
        candidates = np.setdiff1d(c, I_u, assume_unique=True)
        return U_u, candidates

    def get_mean_by_user(self, user_id):
        user_ratings = self.feedbacks[self.feedbacks[:, 0] == self.user_id][:, 3]
        mean_for_user = np.mean(user_ratings) if len(user_ratings) > 0 else 0
        return mean_for_user

    def similarity_with_means(self, item_id: int, U_u: np.array) -> float:
        mean_user = self.get_mean_by_user(self.user_id)
        similarity = 0
        for uid in U_u:
            neighbor_rating_item = self.feedbacks[(self.feedbacks[:, 0] == uid) & (self.feedbacks[:, 1] == item_id)][:, 3]
            if neighbor_rating_item.size > 0:
                mean_neigbor = self.get_mean_by_user(uid)
                similarity += self.similarities[self.user_id, uid] * (neighbor_rating_item - mean_neigbor)
        abs_similarity = np.abs(self.similarities[self.user_id, :]).sum()
        return mean_user + similarity / abs_similarity

    def similarity_without_means(self, item_id: int, U_u: np.array) -> float:
        similarity = 0
        for uid in U_u:
            neighbor_rating_item = self.feedbacks[(self.feedbacks[:, 0] == uid) & (self.feedbacks[:, 1] == item_id)][:, 3]
            if neighbor_rating_item.size > 0:
                similarity += self.similarities[self.user_id, uid] * neighbor_rating_item
        abs_similarity = np.abs(self.similarities[self.user_id, :]).sum()
        return similarity / abs_similarity if abs_similarity > 0 else 0

    def rank_candidates(self, candidates: np.array, U_u: np.array) -> List:
        sims = [self.similarity_with_means(c, U_u) if self.mean_correct \
                else self.similarity_without_means(c, U_u) for c in candidates]
        candidates = iencoder_train.inverse_transform(candidates)
        mapping = list(zip(candidates, sims))
        return sorted(mapping, key=lambda couple: couple[1], reverse=True)

    def topn_recommendation(self, userId: int, k: int = -1, N=30) -> pd.DataFrame:
        U_u, candidates = self.candidate_items(userId, k)
        ranked_candidates = self.rank_candidates(candidates, U_u)
        topn = pd.DataFrame(ranked_candidates[:N], columns=['itemid','similarity_with_Iu'])
        topn = pd.merge(topn, self.movies, on='itemid', how='inner')
        topn['similarity_with_Iu'] = topn['similarity_with_Iu'].apply(lambda x: round(x[0], 3))
        return topn

    def fit(self, feedbacks: np.array):
        self.feedbacks = feedbacks.to_numpy()
        self.similarities, self.neighbors = self.calculate_pairwise_sim()

    def recommend(self, user: int, n: int):
        self.user_id = user
        return self.topn_recommendation(userId=self.user_id, k=10, N=n)

This way you have got 6 different recommendation methods (each of two CF modes can be used with 3 similarity scores).

## Task 3: Apply models

1. For all 6 possible algorithm variations train it and compute recomendations for validation part. (10 points)

In [11]:
model = User2UserRecomendations(SimilarityFunctions.sim_dot, movies, mean_correct=False)
res = model.fit(data_train)

  0%|          | 0/444153 [00:00<?, ?it/s]

In [16]:
model.recommend(163, 10)

,itemid,similarity_with_Iu,title
0,763,0.007,Happy Gilmore (1996)
1,98,0.005,"Silence of the Lambs, The (1991)"
2,50,0.005,Star Wars (1977)
3,79,0.004,"Fugitive, The (1993)"
4,257,0.004,Men in Black (1997)
5,22,0.004,Braveheart (1995)
6,288,0.004,Scream (1996)
7,42,0.004,Clerks (1994)
8,219,0.004,"Nightmare on Elm Street, A (1984)"
9,1,0.004,Toy Story (1995)


In [13]:
model1 = User2UserRecomendations(SimilarityFunctions.sim_dot, movies, mean_correct=True)
res1 = model1.fit(data_train)

  0%|          | 0/444153 [00:00<?, ?it/s]

In [15]:
model1.recommend(163, 10)

,itemid,similarity_with_Iu,title
0,763,0.007,Happy Gilmore (1996)
1,98,0.005,"Silence of the Lambs, The (1991)"
2,50,0.005,Star Wars (1977)
3,79,0.004,"Fugitive, The (1993)"
4,257,0.004,Men in Black (1997)
5,22,0.004,Braveheart (1995)
6,288,0.004,Scream (1996)
7,42,0.004,Clerks (1994)
8,219,0.004,"Nightmare on Elm Street, A (1984)"
9,1,0.004,Toy Story (1995)


In [17]:
model2 = User2UserRecomendations(SimilarityFunctions.sim_jacc, movies, mean_correct=False)
res2 = model2.fit(data_train)

  0%|          | 0/444153 [00:00<?, ?it/s]

In [18]:
model2.recommend(163, 10)

,itemid,similarity_with_Iu,title
0,272,0.007,Good Will Hunting (1997)
1,50,0.006,Star Wars (1977)
2,302,0.005,L.A. Confidential (1997)
3,1,0.005,Toy Story (1995)
4,147,0.005,"Long Kiss Goodnight, The (1996)"
5,741,0.003,"Last Supper, The (1995)"
6,475,0.003,Trainspotting (1996)
7,15,0.003,Mr. Holland's Opus (1995)
8,273,0.003,Heat (1995)
9,25,0.003,"Birdcage, The (1996)"


In [19]:
model3 = User2UserRecomendations(SimilarityFunctions.sim_jacc, movies, mean_correct=True)
res3 = model3.fit(data_train)

  0%|          | 0/444153 [00:00<?, ?it/s]

In [20]:
model3.recommend(163, 10)

,itemid,similarity_with_Iu,title
0,272,0.007,Good Will Hunting (1997)
1,50,0.006,Star Wars (1977)
2,302,0.005,L.A. Confidential (1997)
3,1,0.005,Toy Story (1995)
4,147,0.005,"Long Kiss Goodnight, The (1996)"
5,741,0.003,"Last Supper, The (1995)"
6,475,0.003,Trainspotting (1996)
7,15,0.003,Mr. Holland's Opus (1995)
8,273,0.003,Heat (1995)
9,25,0.003,"Birdcage, The (1996)"


In [21]:
model4 = User2UserRecomendations(SimilarityFunctions.sim_pearson_decreasing, movies, mean_correct=False)
res4 = model4.fit(data_train)

  0%|          | 0/444153 [00:00<?, ?it/s]

<ipython-input-5-ef6974031f44>:34: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return pearsonr(x, y)[0]


In [ ]:
model5 = User2UserRecomendations(SimilarityFunctions.sim_pearson_decreasing, movies, mean_correct=True)
res5 = model5.fit(data_train)